# Phase 3: 세그멘테이션

이 노트북에서 배울 내용:
- RANSAC을 이용한 평면 추출
- DBSCAN 클러스터링
- 씬 세그멘테이션 파이프라인

In [1]:
import sys
sys.path.append('..')

import numpy as np
import open3d as o3d

from src.data import PointCloudLoader
from src.preprocessing import Preprocessor
from src.segmentation import Segmenter
from src.utils import Visualizer

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## 1. 데이터 준비

In [2]:
loader = PointCloudLoader()
preprocessor = Preprocessor()
segmenter = Segmenter()

# 샘플 씬 생성 및 전처리
scene = loader.create_sample_scene(add_noise=True)
scene = preprocessor.full_pipeline(scene, voxel_size=0.03)

Visualizer.show(scene, "Preprocessed Scene")

샘플 씬 생성: 5,316개 포인트
전처리 파이프라인 시작
다운샘플링: 5,316 → 5,108 (96.1%)
노이즈 제거: 82개 이상치 제거 (1.6%)
법선 추정 완료: 5,026개 법선
전처리 완료: 최종 5,026개 포인트


## 2. RANSAC 평면 추출

**RANSAC (Random Sample Consensus)**

1. 무작위로 3개 점 선택
2. 이 점들로 평면 모델 피팅
3. 모델에 가까운 점(inliers) 수 계산
4. 반복하여 가장 많은 inliers를 가진 모델 선택

**장점**: 이상치(outlier)에 강건함

In [3]:
# 단일 평면 추출
plane, remaining = segmenter.extract_plane(
    scene,
    distance_threshold=0.02,  # 평면까지 최대 거리
    num_iterations=1000       # RANSAC 반복 횟수
)

평면 추출: 2,954개 점
  방정식: 0.000x + 0.000y + 1.000z + -0.001 = 0


In [4]:
# 평면 방정식 해석
a, b, c, d = plane.equation
print(f"평면 방정식: {a:.3f}x + {b:.3f}y + {c:.3f}z + {d:.3f} = 0")
print(f"법선 벡터: ({a:.3f}, {b:.3f}, {c:.3f})")

# 법선이 Z축에 가까우면 바닥/천장
if abs(c) > 0.9:
    print("→ 수평면 (바닥 또는 천장)")
elif abs(a) > 0.9 or abs(b) > 0.9:
    print("→ 수직면 (벽)")

평면 방정식: 0.000x + 0.000y + 1.000z + -0.001 = 0
법선 벡터: (0.000, 0.000, 1.000)
→ 수평면 (바닥 또는 천장)


In [5]:
# 시각화: 평면(회색) + 나머지(원래 색)
Visualizer.show_multiple([plane.points, remaining], "Plane (gray) + Objects")

## 3. 여러 평면 추출

In [6]:
# 순차적으로 여러 평면 추출
planes, objects = segmenter.extract_multiple_planes(
    scene,
    max_planes=2,    # 최대 평면 수
    min_points=100   # 평면당 최소 점 수
)

평면 추출: 2,977개 점
  방정식: 0.000x + -0.001y + 1.000z + -0.000 = 0
평면 추출: 144개 점
  방정식: 0.002x + 0.016y + 1.000z + -0.612 = 0
총 2개 평면 추출, 나머지 1,905개 점


In [7]:
# 각 평면에 다른 색상 적용
colors = [[0.8, 0.8, 0.8], [0.6, 0.6, 0.6]]
geometries = []

for i, plane in enumerate(planes):
    plane.points.paint_uniform_color(colors[i % len(colors)])
    geometries.append(plane.points)

geometries.append(objects)

Visualizer.show_multiple(geometries, "Multiple Planes + Objects")

## 4. DBSCAN 클러스터링

**DBSCAN (Density-Based Spatial Clustering)**

밀도 기반으로 점들을 그룹화합니다.

**파라미터**:
- `eps`: 이웃 판정 거리 (반경)
- `min_points`: 클러스터 최소 점 수

**특징**: 클러스터 개수를 미리 지정할 필요 없음

In [8]:
# 평면 제거 후 남은 점들 클러스터링
clusters = segmenter.cluster_dbscan(
    objects,
    eps=0.1,          # 이웃 거리
    min_points=10     # 최소 클러스터 크기
)

DBSCAN 클러스터링: 22개 클러스터 발견
  클러스터 0: 91개 점, 중심 (0.41, -1.82, 0.42)
  클러스터 1: 29개 점, 중심 (0.87, -1.43, 0.48)
  클러스터 2: 318개 점, 중심 (-1.49, 0.99, 0.31)
  클러스터 3: 57개 점, 중심 (0.34, -1.23, 0.26)
  클러스터 4: 30개 점, 중심 (0.13, -1.50, 0.32)
  클러스터 5: 95개 점, 중심 (0.70, -1.26, 0.39)
  클러스터 6: 18개 점, 중심 (0.35, -1.26, 0.66)
  클러스터 7: 10개 점, 중심 (1.42, 0.20, 0.16)
  클러스터 8: 53개 점, 중심 (1.21, 0.62, 0.32)
  클러스터 9: 42개 점, 중심 (0.34, -1.56, 0.73)
  클러스터 10: 14개 점, 중심 (0.65, -1.34, 0.08)
  클러스터 11: 13개 점, 중심 (0.26, -1.49, 0.10)
  클러스터 12: 28개 점, 중심 (1.78, 0.69, 0.37)
  클러스터 13: 10개 점, 중심 (1.46, 0.80, 0.11)
  클러스터 14: 45개 점, 중심 (1.72, 0.23, 0.17)
  클러스터 15: 6개 점, 중심 (0.59, -1.54, 0.78)
  클러스터 16: 11개 점, 중심 (1.20, 0.26, 0.25)
  클러스터 17: 12개 점, 중심 (1.78, 0.74, 0.07)
  클러스터 18: 8개 점, 중심 (0.27, -1.75, 0.19)
  클러스터 19: 12개 점, 중심 (1.41, 0.80, 0.33)
  클러스터 20: 20개 점, 중심 (0.81, -1.72, 0.36)
  클러스터 21: 12개 점, 중심 (0.66, -1.55, 0.04)


In [9]:
# 클러스터 정보
for cluster in clusters:
    bbox_extent = cluster.bbox.get_extent()
    print(f"클러스터 {cluster.label}:")
    print(f"  점 수: {len(cluster.indices):,}")
    print(f"  중심: ({cluster.centroid[0]:.2f}, {cluster.centroid[1]:.2f}, {cluster.centroid[2]:.2f})")
    print(f"  크기: {bbox_extent[0]:.2f} x {bbox_extent[1]:.2f} x {bbox_extent[2]:.2f}")
    print()

클러스터 0:
  점 수: 91
  중심: (0.41, -1.82, 0.42)
  크기: 0.60 x 0.38 x 0.46

클러스터 1:
  점 수: 29
  중심: (0.87, -1.43, 0.48)
  크기: 0.10 x 0.32 x 0.25

클러스터 2:
  점 수: 318
  중심: (-1.49, 0.99, 0.31)
  크기: 0.61 x 0.62 x 0.56

클러스터 3:
  점 수: 57
  중심: (0.34, -1.23, 0.26)
  크기: 0.37 x 0.30 x 0.54

클러스터 4:
  점 수: 30
  중심: (0.13, -1.50, 0.32)
  크기: 0.13 x 0.29 x 0.36

클러스터 5:
  점 수: 95
  중심: (0.70, -1.26, 0.39)
  크기: 0.38 x 0.42 x 0.69

클러스터 6:
  점 수: 18
  중심: (0.35, -1.26, 0.66)
  크기: 0.14 x 0.21 x 0.20

클러스터 7:
  점 수: 10
  중심: (1.42, 0.20, 0.16)
  크기: 0.16 x 0.04 x 0.15

클러스터 8:
  점 수: 53
  중심: (1.21, 0.62, 0.32)
  크기: 0.12 x 0.48 x 0.49

클러스터 9:
  점 수: 42
  중심: (0.34, -1.56, 0.73)
  크기: 0.33 x 0.44 x 0.18

클러스터 10:
  점 수: 14
  중심: (0.65, -1.34, 0.08)
  크기: 0.18 x 0.22 x 0.11

클러스터 11:
  점 수: 13
  중심: (0.26, -1.49, 0.10)
  크기: 0.09 x 0.28 x 0.10

클러스터 12:
  점 수: 28
  중심: (1.78, 0.69, 0.37)
  크기: 0.13 x 0.32 x 0.22

클러스터 13:
  점 수: 10
  중심: (1.46, 0.80, 0.11)
  크기: 0.16 x 0.04 x 0.18

클러스터 14:
  점 수: 45


In [10]:
# 클러스터 시각화 (각각 다른 색상 + 바운딩 박스)
geometries = []
for cluster in clusters:
    geometries.append(cluster.points)
    geometries.append(cluster.bbox)

Visualizer.show_multiple(geometries, "Clusters with Bounding Boxes")

## 5. eps 파라미터의 영향

In [11]:
# eps 값에 따른 클러스터 수 변화
for eps in [0.05, 0.1, 0.2, 0.3]:
    clusters = segmenter.cluster_dbscan(objects, eps=eps, min_points=10)
    print(f"eps={eps}: {len(clusters)}개 클러스터")

DBSCAN 클러스터링: 0개 클러스터 발견
eps=0.05: 0개 클러스터
DBSCAN 클러스터링: 22개 클러스터 발견
  클러스터 0: 91개 점, 중심 (0.41, -1.82, 0.42)
  클러스터 1: 29개 점, 중심 (0.87, -1.43, 0.48)
  클러스터 2: 318개 점, 중심 (-1.49, 0.99, 0.31)
  클러스터 3: 57개 점, 중심 (0.34, -1.23, 0.26)
  클러스터 4: 30개 점, 중심 (0.13, -1.50, 0.32)
  클러스터 5: 95개 점, 중심 (0.70, -1.26, 0.39)
  클러스터 6: 18개 점, 중심 (0.35, -1.26, 0.66)
  클러스터 7: 10개 점, 중심 (1.42, 0.20, 0.16)
  클러스터 8: 53개 점, 중심 (1.21, 0.62, 0.32)
  클러스터 9: 42개 점, 중심 (0.34, -1.56, 0.73)
  클러스터 10: 14개 점, 중심 (0.65, -1.34, 0.08)
  클러스터 11: 13개 점, 중심 (0.26, -1.49, 0.10)
  클러스터 12: 28개 점, 중심 (1.78, 0.69, 0.37)
  클러스터 13: 10개 점, 중심 (1.46, 0.80, 0.11)
  클러스터 14: 45개 점, 중심 (1.72, 0.23, 0.17)
  클러스터 15: 6개 점, 중심 (0.59, -1.54, 0.78)
  클러스터 16: 11개 점, 중심 (1.20, 0.26, 0.25)
  클러스터 17: 12개 점, 중심 (1.78, 0.74, 0.07)
  클러스터 18: 8개 점, 중심 (0.27, -1.75, 0.19)
  클러스터 19: 12개 점, 중심 (1.41, 0.80, 0.33)
  클러스터 20: 20개 점, 중심 (0.81, -1.72, 0.36)
  클러스터 21: 12개 점, 중심 (0.66, -1.55, 0.04)
eps=0.1: 22개 클러스터
DBSCAN 클러스터링: 4개 클러스터 발견
  클러스

## 6. 전체 씬 세그멘테이션 파이프라인

In [12]:
# 원본 씬 다시 로드
scene = loader.create_sample_scene(add_noise=True)
scene = preprocessor.full_pipeline(scene, voxel_size=0.03)

# 전체 세그멘테이션
planes, clusters = segmenter.segment_scene(scene)

샘플 씬 생성: 5,316개 포인트
전처리 파이프라인 시작
다운샘플링: 5,316 → 5,089 (95.7%)
노이즈 제거: 84개 이상치 제거 (1.7%)
법선 추정 완료: 5,005개 법선
전처리 완료: 최종 5,005개 포인트
씬 세그멘테이션 시작
평면 추출: 2,894개 점
  방정식: -0.001x + 0.001y + 1.000z + -0.001 = 0
평면 추출: 219개 점
  방정식: 0.005x + -0.004y + 1.000z + 0.003 = 0
총 2개 평면 추출, 나머지 1,892개 점
DBSCAN 클러스터링: 24개 클러스터 발견
  클러스터 0: 136개 점, 중심 (0.52, -1.79, 0.37)
  클러스터 1: 56개 점, 중심 (1.22, 0.62, 0.35)
  클러스터 2: 142개 점, 중심 (0.28, -1.31, 0.31)
  클러스터 3: 304개 점, 중심 (-1.50, 1.00, 0.28)
  클러스터 4: 10개 점, 중심 (1.20, 0.40, 0.11)
  클러스터 5: 13개 점, 중심 (1.56, 0.32, 0.60)
  클러스터 6: 43개 점, 중심 (1.69, 0.22, 0.39)
  클러스터 7: 33개 점, 중심 (0.36, -1.49, 0.75)
  클러스터 8: 31개 점, 중심 (1.53, 0.52, 0.60)
  클러스터 9: 19개 점, 중심 (1.79, 0.70, 0.29)
  클러스터 10: 28개 점, 중심 (0.69, -1.71, 0.13)
  클러스터 11: 9개 점, 중심 (0.19, -1.51, 0.16)
  클러스터 12: 31개 점, 중심 (0.62, -1.35, 0.72)
  클러스터 13: 22개 점, 중심 (1.73, 0.78, 0.49)
  클러스터 14: 21개 점, 중심 (1.28, 0.77, 0.51)
  클러스터 15: 15개 점, 중심 (0.78, -1.57, 0.66)
  클러스터 16: 21개 점, 중심 (-1.39, 1.03, 0.56)
  클러스

In [13]:
# 최종 결과 시각화
segmenter.visualize_segmentation(planes, clusters, show_bbox=True)